#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import sys
import pandas as pd
os.environ ['CUDA_VISIBLE_DEVICES'] = "1"

#### 文档加载&切分&检索

In [2]:
mit_info = pd.read_excel("./mit_structure_info.xlsx")
item = mit_info.iloc[0].to_dict()
item.pop("id")
item = {
    '标题': item['name'],
    '简介': item['summary'],
    '文章类别': item['typeName'],
    '文章内容': item['content']
}
item

{'标题': '人类有望在太空生孩子？荷兰公司将把实验室送入近地轨道并进行体外受精',
 '简介': '迄今为止，还没有人在太空中怀孕。但随着太空旅游的兴起，它很可能最终在某一天发生。',
 '文章类别': '太空',
 '文章内容': '是否有可能在太空生孩子？荷兰企业家埃格伯特·埃德布鲁克（Egbert Edelbroek）第一次产生这种想法时，正在捐献精子。埃德布鲁克对被捐赠精子的各种使用方式感到好奇，他开始思考体外受精技术在地球之外是否可行，甚至是否可以借助太空的条件得到改善。太空的失重状态会比实验室培养皿更合适吗？现在，埃德布鲁克是 SpaceBorn United 的 CEO，这是一家生物技术初创公司，旨在开创人类在地球以外繁衍的研究。2024 年，他计划用火箭将一个小型实验室送入近地轨道，并在那里进行体外受精。如果成功，埃德布鲁克希望他的工作能为未来人类建设太空定居点铺平道路。“人类需要一个后备计划，”他说，“如果你想成为一个可持续发展的物种，就要成为多行星物种（multiplanetary species）。”除了未来的太空殖民地，我们还迫切需要了解太空对人类生殖系统的影响。迄今为止，还没有人在太空中怀孕。但随着太空旅游的兴起，它很可能最终在某一天发生。埃德布鲁克认为，我们应该做好准备。尽管埃隆·马斯克（Elon Musk）和杰夫·贝佐斯（Jeff Bezos）在美国国家科学、工程和医学院 9 月发布的一份报告中指出，我们几乎没有对人类在太空中的繁殖进行过研究。并补充说，“我们对太空如何影响繁殖的理解对长期太空探索至关重要，但迄今为止，该方向的研究基本为零。”一些对动物的研究表明，繁殖的各个阶段，从交配和受精到胚胎发育、着床、怀孕和出生，都可以在太空中正常“运作”。例如，1994 年，在第一次相关实验中，八条日本青鳉在哥伦比亚号航天飞机上从卵发育成幼鱼。八条鱼都在返回地球后幸存下来，并且似乎一切正常。循序渐进然而，其他研究也发现了潜在问题的证据。1983 年，怀孕的老鼠在苏联卫星上度过了孕晚期的大部分时间，总共五天，在分娩和分娩过程中出现了并发症。和所有返回地球的宇航员一样，老鼠们疲惫不堪，身体虚弱。它们的分娩持续时间比平时长，可能是因为子宫肌肉萎缩。其中，一窝幼崽全部在分娩过程中死亡，这被认为是由于母亲的虚弱状态造成的。对埃德布鲁克来说，这些不确定

In [3]:
mit_info

,id,name,summary,typeName,content
0,12575,人类有望在太空生孩子？荷兰公司将把实验室送入近地轨道并进行体外受精,迄今为止，还没有人在太空中怀孕。但随着太空旅游的兴起，它很可能最终在某一天发生。,太空,是否有可能在太空生孩子？荷兰企业家埃格伯特·埃德布鲁克（Egbert Edelbroek）第...
1,12573,一篇你不能错过的科普|关于阿秒,阿秒仅仅为10-18秒，如果说光从地球跑到月球的时间大约需要1秒，而在1阿秒时间内，光只能传...,科技,NaN
2,12572,科学家开发纳米加热技术，将器官冷冻保存期延长至100天,手术完成以后，当他看到大鼠的尿液掉落时非常激动，这说明肾能够正常工作，移植手术成功了。,生物医学,“论文的共同一作是一名医生，他来帮我们给大鼠做肾脏移植手术。手术完成以后，当他看到大鼠的尿液...
3,12571,美国每6.5分钟就有一人死于阿片药物过量，三年以来致死人数翻两番,它们与大脑和脊髓中的受体结合，引发一系列有助于阻断疼痛信号的反应。处方阿片类药物在某些情况下...,生物医学,本周，我一直在思考美国人的阿片类药物成瘾问题。统计数字令人震惊。自 2010 年以来，在美国...
4,12570,科学家解析光合玫瑰菌反应中心-捕光复合体结构，助推原核生物光合作用元件研究,他们也发现了类胡萝卜素组装调节反应中心-捕光复合体结构和醌交换的分子机制。,生物科技,“这些高分辨率的结构，清晰地揭示了光合玫瑰菌反应中心-捕光复合体（Reaction cent...
...,...,...,...,...,...
995,11560,​鲁汶大学中国学者发明超高性能混合基质膜，性能超越纯沸石膜,在人类的各项工业生产活动中，化学品分离过程的能源消耗极大，占据全球能源消费总量的 15%，而...,化学,NaN
996,11559,有人用ChatGPT作弊和替考，OpenAI或考虑给AI内容加水印,在人工智能生成的文本中，我们或许可以找到一些隐藏在字里行间的蛛丝马迹。如果有方法可以识别出来...,AI,NaN
997,11558,“冷门”方向坚持30余年，川大王玉忠院士团队研发聚合物燃烧分析系统，造出温光双响应的火灾探测器,2021 年 6 月 17 日，是付腾至今难以忘记的一天。那天，在中国传统村落最多的地级行政...,物理,NaN
998,11556,发光效率达5.12%：科学家实现有机金属卤化物的高效电致发光,在现代生活中，电致发光二极管有着广阔的应用比如显示和照明等。各类发光材料包括外延生长无机半导...,物理,NaN


In [4]:
sys.path.append(os.path.dirname(os.path.realpath('__file__'))+"/../../")
import config.conf as cfg
from chromadb.config import Settings

replace_str = "/spider/mit_spider"

# Define the folder for storing database
PERSIST_DIRECTORY = cfg.PERSIST_DIRECTORY.replace(replace_str,"")
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = cfg.EMBEDDINGS_MODEL_NAME.replace(replace_str,"")
persist_directory = cfg.PERSIST_DIRECTORY.replace(replace_str,"")
model_path = cfg.MODEL_PATH.replace(replace_str,"")
model_n_ctx = cfg.MODEL_N_CTX
model_n_batch = cfg.MODEL_N_BATCH
target_source_chunks = 4
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

In [5]:
cfg.EMBEDDINGS_MODEL_NAME

'/home/linjw/AGI/llama2/spider/mit_spider/checkpoints/paraphrase-multilingual-MiniLM-L12-v2'

#### 加载模型

In [6]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id=model_path,
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 16384, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 50,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


#### 回答模板

In [7]:
que = str(item)
que

"{'标题': '人类有望在太空生孩子？荷兰公司将把实验室送入近地轨道并进行体外受精', '简介': '迄今为止，还没有人在太空中怀孕。但随着太空旅游的兴起，它很可能最终在某一天发生。', '文章类别': '太空', '文章内容': '是否有可能在太空生孩子？荷兰企业家埃格伯特·埃德布鲁克（Egbert Edelbroek）第一次产生这种想法时，正在捐献精子。埃德布鲁克对被捐赠精子的各种使用方式感到好奇，他开始思考体外受精技术在地球之外是否可行，甚至是否可以借助太空的条件得到改善。太空的失重状态会比实验室培养皿更合适吗？现在，埃德布鲁克是 SpaceBorn United 的 CEO，这是一家生物技术初创公司，旨在开创人类在地球以外繁衍的研究。2024 年，他计划用火箭将一个小型实验室送入近地轨道，并在那里进行体外受精。如果成功，埃德布鲁克希望他的工作能为未来人类建设太空定居点铺平道路。“人类需要一个后备计划，”他说，“如果你想成为一个可持续发展的物种，就要成为多行星物种（multiplanetary species）。”除了未来的太空殖民地，我们还迫切需要了解太空对人类生殖系统的影响。迄今为止，还没有人在太空中怀孕。但随着太空旅游的兴起，它很可能最终在某一天发生。埃德布鲁克认为，我们应该做好准备。尽管埃隆·马斯克（Elon Musk）和杰夫·贝佐斯（Jeff Bezos）在美国国家科学、工程和医学院 9 月发布的一份报告中指出，我们几乎没有对人类在太空中的繁殖进行过研究。并补充说，“我们对太空如何影响繁殖的理解对长期太空探索至关重要，但迄今为止，该方向的研究基本为零。”一些对动物的研究表明，繁殖的各个阶段，从交配和受精到胚胎发育、着床、怀孕和出生，都可以在太空中正常“运作”。例如，1994 年，在第一次相关实验中，八条日本青鳉在哥伦比亚号航天飞机上从卵发育成幼鱼。八条鱼都在返回地球后幸存下来，并且似乎一切正常。循序渐进然而，其他研究也发现了潜在问题的证据。1983 年，怀孕的老鼠在苏联卫星上度过了孕晚期的大部分时间，总共五天，在分娩和分娩过程中出现了并发症。和所有返回地球的宇航员一样，老鼠们疲惫不堪，身体虚弱。它们的分娩持续时间比平时长，可能是因为子宫肌肉萎缩。其中，一窝幼崽全部在分娩过程中死亡，这被认为是由于母亲的虚弱状态造成的。对埃德布鲁克来说，这些不确定的结

In [8]:
# 回答模板 将summary_query的输出格式化为对象
response_schemas = [
    ResponseSchema(name="核心任务", description="给出文章的核心任务, 是有关于公司的主要职责，包括产品开发、市场营销和客户支持等"),
    ResponseSchema(name="创新文化", description="给出项目的创新文化部分,是有关于鼓励提出新点子和方法，以不断改进和发展公司"),
    ResponseSchema(name="激励机制", description="给出激励机制,包括奖励创新和负反馈等方式，以激励创新绩效"),
    ResponseSchema(name="原创技巧", description="给出原创技巧,指对项目的成功至关重要的技巧"),
    ResponseSchema(name="科研兴趣", description="给出科研兴趣,是以推动创新的兴趣部分"),
    ResponseSchema(name="团队协作", description="给出团队协作,是有关该任务由哪些团队协作完成,可以给出具体名字"),
    ResponseSchema(name="发生时间", description="给出文章发生的大概时间"),
    
]

#### 数据解析器

In [9]:
summary_query = """
"
1. 核心任务：文章中涉及到公司的主要职责，包括产品开发、市场营销、客户支持等，以确定文章的核心任务是什么。

2. 创新文化：查找有关鼓励新点子和方法，以不断改进和发展公司的部分，以便了解文章中提到的创新文化。

3. 激励机制：寻找文章中提及的激励机制，包括奖励创新和负反馈等方式，以激励创新绩效。

4. 原创技巧：识别文章中强调的成功至关重要的技巧，这些技巧对项目的成功有何关联。

5. 科研兴趣：分析文章中提到的科研兴趣，以了解它们如何推动创新。

6. 团队协作：查找文章中涉及的团队协作情况，可以提供具体团队的名称或信息。

7. 团队协作：给出文章发生的大概时间。

阅读上面文章，然后提取并分析七个关键信息，每个信息需要包括方法、人名、机构等的具体名字，可以通过全文推理得出。按照要求执行这个任务，返回一个包含七个关键信息的纯Python字典。
"""

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are an article reading analyst. 你是一个文章阅读分析者。\n"
    "<</SYS>>\n\n"
    "以下为文章的主要细节：\n"
    "{context_str}"
    "\n"
    "{format_instructions}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=initial_qa_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"context_str":que}
)
_input = prompt.format_prompt(question=summary_query)
output = model(_input.to_string())
print(output)

 ```python
{
    "核心任务": "为人类在太空中生孩子创造机会",
    "创新文化": "鼓励员工提出新的想法和方法",
    "激励机制": "通过奖金和晋升来奖励创新成果",
    "原创技巧": "深入研究和试验新技术和方法",
    "科研兴趣": "探索太空环境下人体生殖系统的影响",
    "团队协作": "CEO Egbert Edelbroek领导的SpaceBorn United团队",
    "发生时间": "2024年"
}


In [15]:
output_parser.parse(output.replace("```","").replace(" python\n","").replace("\n",""))

{'核心任务': '为人类在太空中生孩子创造机会',
 '创新文化': '鼓励员工提出新的想法和方法',
 '激励机制': '通过奖金和晋升来奖励创新成果',
 '原创技巧': '深入研究和试验新技术和方法',
 '科研兴趣': '探索太空环境下人体生殖系统的影响',
 '团队协作': 'CEO Egbert Edelbroek领导的SpaceBorn United团队',
 '发生时间': '2024年'}